In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull
import laspy

def load_point_cloud(file_path):
    """
    Load 3D point cloud data from a .las file.
    """
    print(f"Loading point cloud from {file_path}")
    
    # Read .las file using laspy
    las = laspy.read(file_path)
    
    # Extract the 3D coordinates (x, y, z)
    points = np.vstack((las.x, las.y, las.z)).transpose()
    
    if points.size == 0:
        raise ValueError("Failed to load point cloud or point cloud is empty")
    
    # Convert points to Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    return pcd

# Preprocess, identify potholes, calculate dimensions, and visualize functions remain the same
# These will operate on the point cloud that is now loaded from .las files

def preprocess_point_cloud(pcd, voxel_size=0.01):
    # Preprocess the point cloud (same as before)
    pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_clean, _ = pcd_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    pcd_clean.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    
    plane_model, inliers = pcd_clean.segment_plane(distance_threshold=0.02, ransac_n=3, num_iterations=1000)
    a, b, c, d = plane_model
    normal = np.array([a, b, c])
    normal = normal / np.linalg.norm(normal)
    z_axis = np.array([0, 0, 1])
    rotation_axis = np.cross(normal, z_axis)
    
    if np.linalg.norm(rotation_axis) > 0:
        rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)
        dot_product = np.dot(normal, z_axis)
        angle = np.arccos(dot_product)
        rotation = o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * angle)
        pcd_clean.rotate(rotation, center=[0, 0, 0])
    
    return pcd_clean

def identify_pothole(pcd, distance_threshold=0.05, min_points=100):
    # Same as before
    plane_model, inliers = pcd.segment_plane(distance_threshold=distance_threshold, 
                                            ransac_n=3, num_iterations=1000)
    road_cloud = pcd.select_by_index(inliers)
    road_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    
    pothole_cloud = pcd.select_by_index(inliers, invert=True)
    
    if len(pothole_cloud.points) < min_points:
        print(f"Insufficient points ({len(pothole_cloud.points)}) for pothole detection")
        return None, road_cloud
    
    points = np.asarray(pothole_cloud.points)
    db = DBSCAN(eps=0.05, min_samples=30).fit(points)
    labels = db.labels_
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    print(f"Identified {n_clusters} potential potholes")
    
    if n_clusters == 0:
        return None, road_cloud
    
    label_counts = np.bincount(labels[labels >= 0])
    largest_cluster = np.argmax(label_counts)
    largest_cluster_indices = np.where(labels == largest_cluster)[0]
    
    pothole_points = points[largest_cluster_indices]
    
    pothole_pcd = o3d.geometry.PointCloud()
    pothole_pcd.points = o3d.utility.Vector3dVector(pothole_points)
    pothole_pcd.paint_uniform_color([1, 0, 0])
    
    return pothole_pcd, road_cloud

def calculate_pothole_dimensions(pothole_pcd, road_pcd):
    # Same as before
    if pothole_pcd is None or len(pothole_pcd.points) == 0:
        print("No valid pothole detected for measurement")
        return None
    
    pothole_points = np.asarray(pothole_pcd.points)
    pothole_z = pothole_points[:, 2]
    pothole_xy = pothole_points[:, 0:2]
    road_points = np.asarray(road_pcd.points)
    pothole_centroid = np.mean(pothole_points, axis=0)
    distances = np.linalg.norm(road_points[:, 0:2] - pothole_centroid[0:2], axis=1)
    nearby_road_indices = np.where(distances < 0.5)[0]
    
    if len(nearby_road_indices) > 0:
        nearby_road_points = road_points[nearby_road_indices]
        road_z_reference = np.mean(nearby_road_points[:, 2])
    else:
        road_z_reference = np.max(road_points[:, 2])
    
    max_depth = road_z_reference - np.min(pothole_z)
    avg_depth = road_z_reference - np.mean(pothole_z)
    
    hull = ConvexHull(pothole_xy)
    hull_points = pothole_xy[hull.vertices]
    
    pcd_2d = o3d.geometry.PointCloud()
    points_2d = np.column_stack((pothole_xy, np.zeros(pothole_xy.shape[0])))
    pcd_2d.points = o3d.utility.Vector3dVector(points_2d)
    obb = pcd_2d.get_axis_aligned_bounding_box()
    
    dimensions = obb.get_extent()  # Use get_extent() instead of obb.extent

    length = max(dimensions[0], dimensions[1])
    width = min(dimensions[0], dimensions[1])
    
    area = hull.volume
    volume = area * avg_depth
    
    dimensions = {
        "max_depth": max_depth,
        "avg_depth": avg_depth,
        "length": length,
        "width": width,
        "area": area,
        "volume": volume
    }
    
    return dimensions

def visualize_results(pcd, road_pcd, pothole_pcd, dimensions):
    # Same as before
    if pothole_pcd is None:
        print("No pothole detected for visualization")
        return
    
    combined_pcd = o3d.geometry.PointCloud()
    combined_pcd.points = o3d.utility.Vector3dVector(
        np.vstack((np.asarray(road_pcd.points), np.asarray(pothole_pcd.points)))
    )
    combined_pcd.colors = o3d.utility.Vector3dVector(
        np.vstack((np.asarray(road_pcd.colors), np.asarray(pothole_pcd.colors)))
    )
    
    print("Visualizing point cloud with detected pothole...")
    o3d.visualization.draw_geometries([combined_pcd], 
                                    window_name="Pothole Detection Results",
                                    width=800, height=600)
    
    if dimensions:
        fig, ax = plt.subplots(figsize=(10, 6))
        
        pothole_points = np.asarray(pothole_pcd.points)
        ax.scatter(pothole_points[:, 0], pothole_points[:, 1], c='red', s=10, alpha=0.6)
        
        road_points = np.asarray(road_pcd.points)
        sample_indices = np.random.choice(len(road_points), min(1000, len(road_points)), replace=False)
        ax.scatter(road_points[sample_indices, 0], road_points[sample_indices, 1], 
                 c='gray', s=3, alpha=0.3)
        
        plt.title(f"Pothole Measurements (Top View)\n"
                f"Max Depth: {dimensions['max_depth']:.2f}m, "
                f"Avg Depth: {dimensions['avg_depth']:.2f}m\n"
                f"Length: {dimensions['length']:.2f}m, "
                f"Width: {dimensions['width']:.2f}m\n"
                f"Area: {dimensions['area']:.2f}m², "
                f"Volume: {dimensions['volume']:.2f}m³")
        
        plt.axis('equal')
        plt.grid(True)
        plt.xlabel('X (meters)')
        plt.ylabel('Y (meters)')
        plt.tight_layout()
        plt.show()

def analyze_pothole(file_path):
    """
    Main function to analyze a pothole from a .las point cloud file
    """
    pcd = load_point_cloud(file_path)
    pcd_processed = preprocess_point_cloud(pcd)
    pothole_pcd, road_pcd = identify_pothole(pcd_processed)
    
    if pothole_pcd is None:
        print("No pothole detected in the point cloud")
        return
    
    dimensions = calculate_pothole_dimensions(pothole_pcd, road_pcd)
    
    if dimensions:
        print("\nPothole Dimensions:")
        print(f"Maximum Depth: {dimensions['max_depth']:.3f} meters")
        print(f"Average Depth: {dimensions['avg_depth']:.3f} meters")
        print(f"Length: {dimensions['length']:.3f} meters")
        print(f"Width: {dimensions['width']:.3f} meters")
        print(f"Surface Area: {dimensions['area']:.3f} square meters")
        print(f"Approximate Volume: {dimensions['volume']:.3f} cubic meters")
    
    visualize_results(pcd, road_pcd, pothole_pcd, dimensions)

# Example usage
if __name__ == "__main__":
    pothole_point_cloud_file = r"C:\Users\Shashank\OneDrive\Desktop\Pot\Untitled_Scan_13_13_09.las"  # Replace with your .las file path
    analyze_pothole(pothole_point_cloud_file)


Loading point cloud from C:\Users\Shashank\OneDrive\Desktop\Pot\Untitled_Scan_13_13_09.las
Identified 2 potential potholes


AttributeError: 'open3d.cpu.pybind.geometry.AxisAlignedBoundingBox' object has no attribute 'extent'